# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.environment import Environment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core import Model

from azureml.train.automl.utilities import get_primary_metrics
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.automl.core.shared import constants

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.28.0


## Predicting presence of Amphibians using GIS and satellite images

### Initialize Workspace

In [2]:
ws = Workspace.from_config()

In [3]:
print('Workspace name:  ' + ws.name, 
      'Azure region:    ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group:  ' + ws.resource_group, sep = '\n')

Workspace name:  quick-starts-ws-147168
Azure region:    southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group:  aml-quickstarts-147168


### Azure ML experiment

In [4]:
# choose a name for experiment
experiment_name = 'AR-Capstone-Automl'
project_folder = './automl-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
AR-Capstone-Automl,quick-starts-ws-147168,Link to Azure Machine Learning studio,Link to Documentation


## Compute cluster

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ar-compute"

try:
    compute_cluster = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing compute cluster')
    
except ComputeTargetException:
    print('Creating new compute cluster ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=5)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Creating new compute cluster ...
Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

### Overview

The goal of the project is to predict the presence of amphibians species near the water reservoirs based on features obtained from GIS and satellite images.

The dataset used for the project is the **Amphibians Data Set** available from Machine Learning Repository at University of California, Irvine.

The dataset is multilabel and can be used to predict presence of a number of amphibian species.

In this project we will be using the dataset to predict the presence of **Green frogs** (Label 1).

The link to the original dataset is: http://archive.ics.uci.edu/ml/datasets/Amphibians

### Amphibians Data Set

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Amphibians Dataset"
description_text = "Amphibians Dataset for Udacity Capstone project"

if key in ws.datasets.keys(): 
        print('Found existing dataset')
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create Dataset and register it in Workspace
        print('Creating dataset ...')
        data_loc = "https://raw.githubusercontent.com/arees88/ML_Azure_P3_Capstone/main/Amphibians_dataset_green_frogs.csv"
        dataset = Dataset.Tabular.from_delimited_files(data_loc)
        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Creating dataset ...


,SR,NR,TR,VR,SUR1,SUR2,SUR3,UR,FR,OR,RR,BR,MR,CR,Label1
count,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.00000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000
mean,9633.227513,1.566138,4.952381,1.904762,4.232804,5.391534,5.84127,0.841270,0.846561,89.962963,2.333333,2.502646,0.047619,1.015873,0.571429
std,46256.078309,1.544419,5.590918,1.317407,3.434615,3.515185,3.29348,1.315291,1.349843,19.904926,2.520132,2.640971,0.296894,0.125316,0.496186
min,30.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.00000,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,300.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.00000,0.000000,0.000000,99.000000,1.000000,1.000000,0.000000,1.000000,0.000000
50%,700.000000,1.000000,1.000000,2.000000,2.000000,6.000000,6.00000,0.000000,0.000000,100.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,3300.000000,1.000000,12.000000,3.000000,7.000000,9.000000,9.00000,3.000000,2.000000,100.000000,5.000000,5.000000,0.000000,1.000000,1.000000
max,500000.000000,12.000000,15.000000,4.000000,14.000000,11.000000,11.00000,3.000000,4.000000,100.000000,10.000000,10.000000,2.000000,2.000000,1.000000


## AutoML Configuration

Explain why you chose the automl settings and cofiguration you used below.

Setting early termination to True and the timeout to 30 min to limit AutoML duration.
Setting the maximum iterations concurrency to 4, as the maximum nodes configured in the compute cluster must be greater than the number of concurrent operations in the experiment, and the compute cluster has 5 nodes configured. 
I selected accuracy as the primary metric. The AutoML will perform 4 cross validations.

In [7]:
# automl settings 
automl_settings = {
    "enable_early_stopping": True,
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "featurization": 'auto',
    "primary_metric": 'accuracy',
    "verbosity": logging.INFO,
    "n_cross_validations": 4
}

# Automl config 
automl_config = AutoMLConfig(  
                    task = "classification",
                    compute_target = compute_cluster, 
                    training_data = dataset,
                    label_column_name = "Label1",   
                    debug_log = "automl_errors.log",
                    enable_onnx_compatible_models = True,
                    **automl_settings  
)

In [8]:
# Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on ar-compute with default configuration
Running on remote compute: ar-compute


Experiment,Id,Type,Status,Details Page,Docs Page
AR-Capstone-Automl,AutoML_2886a22c-d991-455c-93ec-194bd8969aaf,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
AR-Capstone-Automl,AutoML_2886a22c-d991-455c-93ec-194bd8969aaf,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_2886a22c-d991-455c-93ec-194bd8969aaf',
 'target': 'ar-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-06-14T17:39:32.909418Z',
 'endTimeUtc': '2021-06-14T18:08:09.529527Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'ar-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"AR-Capstone-Automl","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-147168","workspace_name":"quick-starts-ws-147168","region":"southcentralus","compute_target":"ar-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"accuracy",

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



### Save the Best model

In [11]:
# Retrieve the best automl model
best_run, best_model = remote_run.get_output()

In [14]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AR-Capstone-Automl,AutoML_2886a22c-d991-455c-93ec-194bd8969aaf_58,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
print('Best model Run Id:          ' + str(best_run.get_details()['runId']))
print('Best model Name:            ' + str(best_run.properties['model_name']))
print('Best model Accurecy:        ' + str(best_run.get_metrics()['accuracy']))

Best model Run Id:          AutoML_2886a22c-d991-455c-93ec-194bd8969aaf_58
Best model Name:            AutoML2886a22cd58
Best model Accurecy:        0.7457890070921986


In [16]:
print(best_run)

Run(Experiment: AR-Capstone-Automl,
Id: AutoML_2886a22c-d991-455c-93ec-194bd8969aaf_58,
Type: azureml.scriptrun,
Status: Completed)


In [17]:
print(best_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    weight_column_name=None,
    cv_split_column_names=None,
    enable_streaming=None,
    timeseries_param_dict=None,
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0.3125, reg_lambda=0.4166666666666667, subsample=0.5, tree_method='hist'))], verbose=False))], flatten_transform=None, weights=[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]))],
         verbose=False)


#### Best metrics

In [19]:
best_run_metrics = best_run.get_metrics() 

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    # print(metric_name, metric)
    print("{:33}:  {}".format(metric_name, metric))

balanced_accuracy                :  0.7368912377533068
weighted_accuracy                :  0.7542348962912881
f1_score_micro                   :  0.7457890070921986
precision_score_macro            :  0.7468130691464733
AUC_macro                        :  0.7808941537389813
norm_macro_recall                :  0.4737824755066134
accuracy                         :  0.7457890070921986
precision_score_micro            :  0.7457890070921986
matthews_correlation             :  0.48345758185324794
recall_score_micro               :  0.7457890070921986
AUC_micro                        :  0.7869421691879182
precision_score_weighted         :  0.7520840535178757
average_precision_score_micro    :  0.7931578028154541
f1_score_weighted                :  0.7446292948637574
average_precision_score_weighted :  0.79666626792115
log_loss                         :  0.5676111827808709
recall_score_weighted            :  0.7457890070921986
average_precision_score_macro    :  0.7874704802909486
f1_score_ma

#### Register best model

In [21]:
reg_model = best_run.register_model(model_path='outputs/model.pkl', model_name='automl_best_model',
                        properties={'Accuracy': best_run_metrics['accuracy'], 'AUC': best_run_metrics['AUC_weighted']})

#### Download best model

In [22]:
import os
out_dir = 'download_automl'
if not os.path.isdir(out_dir):
    print ("Creating {:} ...".format(out_dir))
    os.mkdir(out_dir)

Creating download_automl ...


In [23]:
reg_model.download(target_dir = "./download_automl/", exist_ok=True)

'download_automl/model.pkl'

#### Save model

In [24]:
import os
out_dir = 'outputs'
if not os.path.isdir(out_dir):
    print ("Creating {:} ...".format(out_dir))
    os.mkdir(out_dir)

Creating outputs ...


In [25]:
import joblib
joblib.dump(best_model, filename = "outputs/automl_best_model.pkl")

['outputs/automl_best_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

### Register the Model

In the cell below, register the model, create an inference config and deploy the model as a web service.

#### Download env file

In [26]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'azure_scoring.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'azure_env.yml')

#### List registered models

In [27]:
from azureml.core import Model

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)

automl_best_model version: 1
	 Accuracy : 0.7457890070921986
	 AUC : 0.7808941537389814


#### Download env file

In [28]:
# Download env file
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'automl_env.yml')

In [29]:
# Create env from Conda specification file
myenv = Environment.from_conda_specification(name = "myenv", file_path = "./automl_env.yml")

In [30]:
myenv

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210301.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
        

#### List cureted environments

In [31]:
# Microsoft Reference
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments

# List the curated environments and their packages
# from azureml.core.environment import Environment
# envs = Environment.list(workspace=ws)
# for env in envs:
#     if env.startswith("AzureML"):
#         print("Name",env)
#         print("packages", envs[env].python.conda_dependencies.serialize_to_string())

#### Deploy model

In [29]:
# Deploy model

from azureml.core.model import InferenceConfig

inference_config  = InferenceConfig(entry_script = "score.py", environment = myenv)

deployment_config = AciWebservice.deploy_configuration( cpu_cores = 1, 
                                                        memory_gb = 1, 
                                                        enable_app_insights = True,
                                                        description = 'Predict presence of amphibians - AutoML model')

model = Model(ws, 'automl_best_model')

service = Model.deploy(workspace = ws, 
                       name = 'amphibians-predict-automl', 
                       models = [model], 
                       inference_config = inference_config, 
                       deployment_config = deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-12 23:49:24+00:00 Creating Container Registry if not exists.
2021-06-12 23:49:24+00:00 Registering the environment.
2021-06-12 23:49:25+00:00 Building image..
2021-06-13 00:02:33+00:00 Generating deployment configuration.
2021-06-13 00:02:34+00:00 Submitting deployment to compute..
2021-06-13 00:02:37+00:00 Checking the status of deployment amphibians-predict-automl..
2021-06-13 00:06:44+00:00 Checking the status of inference endpoint amphibians-predict-automl.
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Send request to Web Service 

In the cell below, send a request to the web service you deployed to test it.

In [32]:
# Enable Application Insights
service.update(enable_app_insights = True)

In [34]:
print("Service state: " + service.state)
print("Swagger URI:   " + service.swagger_uri)
print("Scoring URI:   " + service.scoring_uri)

Service state: Healthy
Swagger URI:   http://4c070f13-3bf9-43f4-9e49-4d932edfd971.southcentralus.azurecontainer.io/swagger.json
Scoring URI:   http://4c070f13-3bf9-43f4-9e49-4d932edfd971.southcentralus.azurecontainer.io/score


#### Request

In [36]:
import requests
import json

data = {
      "data": [
        {
          "SR": 7000,
          "NR": 1,
          "TR": 1,
          "VR": 1,
          "SUR1": 2,
          "SUR2": 6,
          "SUR3": 10,
          "UR": 3,
          "FR": 3,
          "OR": 50,
          "RR": 1,
          "BR": 5,
          "MR": 0,
          "CR": 1
        },
        {
          "SR": 250,
          "NR": 1,
          "TR": 5,
          "VR": 0,
          "SUR1": 6,
          "SUR2": 8,
          "SUR3": 2,
          "UR": 3,
          "FR": 4,
          "OR": 15,
          "RR": 0,
          "BR": 0,
          "MR": 0,
          "CR": 1
        },
        {
          "SR": 1700,
          "NR": 1,
          "TR": 1,
          "VR": 1,
          "SUR1": 2,
          "SUR2": 6,
          "SUR3": 10,
          "UR": 0,
          "FR": 1,
          "OR": 100,
          "RR": 2,
          "BR": 2,
          "MR": 2,
          "CR": 1
        },
        {
          "SR": 30000,
          "NR": 1,
          "TR": 1,
          "VR": 3,
          "SUR1": 1,
          "SUR2": 1,
          "SUR3": 1,
          "UR": 0,
          "FR": 0,
          "OR": 100,
          "RR": 2,
          "BR": 10,
          "MR": 0,
          "CR": 1
        }   
    ]
}

# Convert to JSON string
input_data = json.dumps(data)

with open("input_data_2.json", "w") as f:
    f.write(input_data)
              
# Set the content type to json
headers = {'Content-Type': 'application/json'}

# Make the request and display the result
result = requests.post(service.scoring_uri, input_data, headers=headers)
print(result.json())

[1, 0, 1, 1]


### Logs

In the cell below, print the logs of the web service and delete the service

In [35]:
# Print the logs of the deployed service
print(service.get_logs())

2021-06-13T00:06:38,430516800+00:00 - rsyslog/run 
rsyslogd: /azureml-envs/azureml_9f0c3287c829596c71d31603ecf0cccf/lib/libuuid.so.1: no version information available (required by rsyslogd)
2021-06-13T00:06:38,465690900+00:00 - nginx/run 
2021-06-13T00:06:38,472118500+00:00 - gunicorn/run 
2021-06-13T00:06:38,490567400+00:00 - iot-server/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-06-13T00:06:38,838046100+00:00 - iot-server/finish 1 0
2021-06-13T00:06:38,844979100+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (69)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 98
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-06-13 00:06:44,513 | root | INFO | Starting up app insights client
2021-06-13 00:06:44,514 | root | INFO | Starting up request id generator
2021-06-13 00:06

## Save model in ONNX format

### Retrieve the Best ONNX Model

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

Set the parameter return_onnx_model=True to retrieve the best ONNX model, instead of the Python model.

In [32]:
best_run, onnx_model = remote_run.get_output(return_onnx_model=True)

In [33]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AR-Capstone-Automl,AutoML_2886a22c-d991-455c-93ec-194bd8969aaf_58,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [1]:
# onnx_model

### Save the Best ONNX Model

In [35]:
out_dir = 'onnx'
if not os.path.isdir(out_dir):
    print ("Creating {:} ...".format(out_dir))
    os.mkdir(out_dir)

Creating onnx ...


In [38]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_path = out_dir + "/automl_best_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, onnx_path)

In [39]:
# Check saved model size
print(os.path.getsize(onnx_path))

615846


### Test data

In [40]:
test_data = "https://raw.githubusercontent.com/arees88/ML_Azure_P3_Capstone/main/Amphibians_testset.csv"
# test_data = "./Amphibians_testset.csv"
test_dataset = Dataset.Tabular.from_delimited_files(test_data)

In [41]:
test_dataset

{
  "source": [
    "https://raw.githubusercontent.com/arees88/ML_Azure_P3_Capstone/main/Amphibians_testset.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [42]:
df = test_dataset.to_pandas_dataframe()
df

,SR,NR,TR,VR,SUR1,SUR2,SUR3,UR,FR,OR,RR,BR,MR,CR,Label1
0,7000,1,1,1,2,6,10,3,3,50,1,5,0,1,1
1,250,1,5,0,6,8,2,3,4,15,0,0,0,1,0
2,1700,1,1,1,2,6,10,0,1,100,2,2,2,1,1
3,30000,1,1,3,1,1,1,0,0,100,2,10,0,1,1


### Predict with ONNX Model

#### Using onnxruntime package

In [43]:
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False

import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    test_df = test_dataset.to_pandas_dataframe()
    mdl_bytes = onnx_model.SerializeToString()
    onnx_res = get_onnx_res(best_run)

    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(test_df)

    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Please use Python version 3.6 or 3.7 to run the inference helper.')

[1 0 1 1]
[[0.1954983  0.80450165]
 [0.57295805 0.42704195]
 [0.34642792 0.65357214]
 [0.23338166 0.7666183 ]]


### Delete Web Service

In [ ]:
# Delete the deployed service
service.delete()